In [2]:
# -*- coding: utf-8 -*-
# ==========================================================
# Clean "Ngarkesa Totale" in ost_data.csv  ->  ost_data1.csv
# - Handles H24 -> next day 00:00
# - Replaces negative "Ngarkesa Totale" by per-hour rolling median (±14d; window=29, center=True)
# - Time-aware linear interpolation for continuity
# - Reconstructs full hourly timeline; writes cleaned CSV in the same folder
# ==========================================================

from pathlib import Path
import numpy as np
import pandas as pd
import sys
import re

REQUIRED_COLS = {"Data", "Ora", "Ngarkesa Totale"}

def here_dir() -> Path:
    """Kthen folderin ku ndodhet skripta; fallback te cwd në Jupyter."""
    try:
        return Path(__file__).resolve().parent  # kur ekzekutohet si .py
    except NameError:
        return Path.cwd()                       # Jupyter / IPython

def read_input_csv(path: Path) -> pd.DataFrame:
    """
    Lexon CSV me autodetektim ndarësi (',' ose ';').
    Ruaj tiparet bazë: heq hapësirat në emrat e kolonave.
    """
    # autodetect sep
    try:
        df = pd.read_csv(path, sep=None, engine="python")
    except Exception:
        # fallback te ',' nëse autodetect dështon
        df = pd.read_csv(path)

    df.columns = df.columns.str.strip()
    missing = REQUIRED_COLS - set(df.columns)
    if missing:
        raise ValueError(
            f"Skedari duhet të përmbajë kolonat: {sorted(REQUIRED_COLS)}. Mungojnë: {sorted(missing)}"
        )
    return df

def build_datetime_index(df: pd.DataFrame) -> pd.DataFrame:
    """
    Krijon indeksin kohor:
      - 'Data' -> datetime
      - 'Ora' pranon format si 'H05', 'H5', '05', '5', 'H24' etj.
      - H24 zhvendoset në ditën pasuese, ora 00
    Kthen një kopje me indeks Datetime të renditur dhe orë të plota (reindex).
    """
    work = df.copy()

    # Parse 'Data'
    work["Data"] = pd.to_datetime(work["Data"], errors="coerce")

    # Nxirr vetëm shifrat nga 'Ora'
    ora_str = work["Ora"].astype(str)
    ora_num = ora_str.str.extract(r"(\d+)", expand=False).astype(float)  # përdor float për të toleruar NaN
    if ora_num.isna().any():
        raise ValueError("Disa vlera te 'Ora' nuk mund të parse-ohen. Prisni format si H00..H24 ose 0..24.")

    ora_num = ora_num.astype(int)

    # H24 -> në ditën tjetër, ora 00
    is_24 = ora_num == 24
    work.loc[is_24, "Data"] = work.loc[is_24, "Data"] + pd.Timedelta(days=1)
    ora_num = ora_num.where(~is_24, 0)

    # Ndërto Datetime
    dt = pd.to_datetime(
        work["Data"].dt.strftime("%Y-%m-%d") + " " + ora_num.astype(str).str.zfill(2),
        format="%Y-%m-%d %H",
        errors="coerce",
    )
    work.insert(0, "Datetime", dt)
    work = work.set_index("Datetime").sort_index()
    work = work[~work.index.isna()].copy()

    # Reindex në orë të plota në intervalin [min, max]
    full_index = pd.date_range(work.index.min(), work.index.max(), freq="H")
    work = work.reindex(full_index)

    # Siguro që kolona e ngarkesës është numerike
    work["Ngarkesa Totale"] = pd.to_numeric(work["Ngarkesa Totale"], errors="coerce")

    return work

def per_hour_center_median(series: pd.Series, window_len: int = 29, min_pts: int = 3) -> pd.Series:
    """Medianë lëvizëse e qendruar mbi seri (për orë të njëjta të ditës)."""
    return series.rolling(window=window_len, center=True, min_periods=min_pts).median()

def fix_load(work: pd.DataFrame) -> pd.DataFrame:
    """
    Rregullon 'Ngarkesa Totale':
      - Zëvendëson vlerat negative me medianën e lëvizshme të orës përkatëse
      - Interpolim kohor + mbushje përpara/pas
      - Siguri për vlera <= 0
    """
    load_col = "Ngarkesa Totale"
    work = work.copy()
    work["Load_raw"] = work[load_col]

    # Maskë për vlera negative
    neg_mask = work["Load_raw"] < 0

    # Medianë orare (grupim sipas orës së ditës, 0..23), dritare qendrore ~ ±14 ditë
    rolling_median = (
        work["Load_raw"]
        .groupby(work.index.hour)
        .apply(per_hour_center_median)
        .reset_index(level=0, drop=True)
    )
    work["RollingMedian_sameHour"] = rolling_median

    # Zëvendëso NEGATIVET me medianën e orës (nëse mungon medianë, lëri NaN që të interp.)
    work["Load_fix"] = work["Load_raw"]
    work.loc[neg_mask, "Load_fix"] = work.loc[neg_mask, "RollingMedian_sameHour"]

    # Interpolim kohor + mbushje sigurie
    work["Load_fix"] = work["Load_fix"].interpolate(method="time", limit_direction="both")
    work["Load_fix"] = work["Load_fix"].fillna(method="bfill").fillna(method="ffill")

    # Rrjet sigurie: nëse mbetet <= 0, kthe në NaN dhe interp sërish
    still_bad = work["Load_fix"] <= 0
    if still_bad.any():
        work.loc[still_bad, "Load_fix"] = np.nan
        work["Load_fix"] = work["Load_fix"].interpolate(method="time", limit_direction="both")
        work["Load_fix"] = work["Load_fix"].fillna(method="bfill").fillna(method="ffill")

    # Vendos kolonën e pastruar
    work[load_col] = work["Load_fix"]

    return work

def reconstruct_output(work: pd.DataFrame, original_cols: list[str]) -> pd.DataFrame:
    """
    Rikthen Data/Ora nga indeksi në formatin origjinal:
      - 'Data' = YYYY-MM-DD
      - 'Ora'  = HXX
    Ruaj të gjitha kolonat origjinale (nëse shtohen orë të reja, ato kolona mbeten NaN),
    por 'Ngarkesa Totale' merr vlerën e pastruar.
    """
    out = work.copy()

    # Rindërto 'Data' dhe 'Ora' nga indeksi
    out["Data"] = out.index.date.astype("datetime64[ns]")
    out["Ora"] = "H" + out.index.hour.astype(str).str.zfill(2)

    # Vendos kolonat sipas rendit origjinal, duke i mbajtur nëse ekzistojnë
    cols = list(original_cols)
    # Sigurohu që 'Data' dhe 'Ora' janë në pozicionet e tyre (nëse mungojnë te original_cols, shtoji)
    if "Data" not in cols:
        cols.insert(0, "Data")
    if "Ora" not in cols:
        cols.insert(1, "Ora")

    # Bashko me kolonat e tjera që mund të jenë shtuar gjatë procesit (p.sh. ndonjë ndihmëse)
    cols = [c for c in cols if c in out.columns]

    out_df = out[cols].copy()
    # Sigurohu që "Ngarkesa Totale" është brenda out_df
    if "Ngarkesa Totale" not in out_df.columns and "Ngarkesa Totale" in out.columns:
        out_df["Ngarkesa Totale"] = out["Ngarkesa Totale"]

    # Rendit sipas kohës (tashmë është i renditur), reset index
    out_df = out_df.reset_index(drop=True)
    return out_df

def main():
    base = here_dir()
    in_path = base / "ost_data.csv"
    out_path = base / "ost_data_clean.csv"

    if not in_path.exists():
        raise FileNotFoundError(f"Nuk u gjet hyrja: {in_path}")

    # 1) Lexo hyrjen
    df = read_input_csv(in_path)

    # 2) Ndërto indeksin kohor me rregullin H24 -> 00 të ditës pasuese
    work = build_datetime_index(df)

    # 3) Pastrim i 'Ngarkesa Totale'
    work_clean = fix_load(work)

    # 4) Rikonstruktim i daljes me Data/Ora dhe kolonat origjinale
    out_df = reconstruct_output(work_clean, original_cols=list(df.columns))

    # 5) Shkruaj CSV (ndarës standard ','); përdor sep=';' nëse preferohet pikëpresja
    out_df.to_csv(out_path, index=False, encoding="utf-8", sep=";")
    # Për ndarës ';', zëvendëso me:
    # out_df.to_csv(out_path, index=False, encoding="utf-8", sep=";")

    neg_count = int((work["Load_raw"] < 0).sum()) if "Load_raw" in work.columns else int((work["Ngarkesa Totale"] < 0).sum(skipna=True))
    print(f"✔ U krijua skedari i pastruar: {out_path}")
    print(f"Rreshta negativë të korrigjuar (përpara interpolimit): {neg_count}")

if __name__ == "__main__":
    # Rekomandohet: `python clean_ost_data.py`
    # ose në Jupyter: kopjoje qelizën dhe ekzekuto `main()`
    main()
        



✔ U krijua skedari i pastruar: C:\Users\Alketa\Artikull OST\OST _publikim\ost_data_clean.csv
Rreshta negativë të korrigjuar (përpara interpolimit): 5


C:\Users\Alketa\AppData\Local\Temp\ipykernel_14096\1805534491.py:82: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_index = pd.date_range(work.index.min(), work.index.max(), freq="H")
C:\Users\Alketa\AppData\Local\Temp\ipykernel_14096\1805534491.py:123: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  work["Load_fix"] = work["Load_fix"].fillna(method="bfill").fillna(method="ffill")
